In [ ]:
import sys
!pip install igraph

#Importing Drive
runningOnColab = 'google.colab' in sys.modules
if runningOnColab:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/Shareddrives/ECE232E/Project 2'

     |████████████████████████████████| 3.1 MB 4.4 MB/s 
Mounted at /content/drive
/content/drive/Shareddrives/ECE232E/Project 2


In [ ]:
import numpy as np
import os
import igraph
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from numpy import dot
from numpy.linalg import norm
from igraph import Graph
import math

from collections import Counter
import matplotlib.pyplot as plt
!unzip "Cora.zip"

Archive:  Cora.zip
replace ._Cora? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ._Cora                  
  inflating: Cora/._edges.csv        
  inflating: Cora/edges.csv          
  inflating: Cora/._group-edges.csv  
  inflating: Cora/group-edges.csv    


In [ ]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    #print(root)
    #print(dirs)
    #print(files)
    #print('--------------------------------')
    for file in files:
        print(file)
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

print(all_data[0])                
#random_state = 42
#all_data = shuffle(all_data,random_state=random_state)

README
cora.cites
cora.content
31336	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

In [ ]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']
sorted(categories)
label_encoder = {}
i = 0
for cat in sorted(categories):
  label_encoder[cat] = i
  i +=1
label_encoder


{'Case_Based': 0,
 'Genetic_Algorithms': 1,
 'Neural_Networks': 2,
 'Probabilistic_Methods': 3,
 'Reinforcement_Learning': 4,
 'Rule_Learning': 5,
 'Theory': 6}

In [ ]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    #print(X)
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i

X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes

F = X.shape[1] #the size of node features
print('X shape: ', X.shape)


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Graph info: ', nx.info(G))

Graph info:  Graph with 2708 nodes and 5278 edges


In [ ]:
nodes = list(G.nodes)
print(len(nodes))
list(G.neighbors(0))

2708


[258, 544, 8, 435, 14]

In [ ]:
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)
#print(gcc_nodes)

In [ ]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes 
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [ ]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g)
    transition_matrix = adj/adj.sum(axis=1)

    return transition_matrix

In [ ]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank

def random_walk(g, num_steps, start_node, transition_matrix = None):
    if transition_matrix is None:
        transition_matrix = create_transition_matrix(g)

    v = start_node
    walked_nodes = range(transition_matrix.shape[0])

    for i in range(num_steps):
        PMF = transition_matrix[v,]
        v = np.random.choice(walked_nodes, p = np.ravel(PMF, order = 'C'))

    return v


In [ ]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node,tp,predicted, transition_matrix = None):
    if transition_matrix is None:
        transition_matrix = create_transition_matrix(g)

    v = start_node
    walked_nodes = range(transition_matrix.shape[0])
    seeds = seeds_dict[predicted]

    for i in range(num_steps):
        run = np.random.uniform(low=0.0, high=1.0, size=None)
        if run < tp:
            v = np.random.choice(seeds)
        else:
            PMF = transition_matrix[v,]
            v = np.random.choice(walked_nodes, p = np.ravel(PMF, order='C'))

    return v




In [ ]:
#pagerank. NO teleportation, NO tfidf. 
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]


for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  for i in range(num_samples):
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      end_node = int(end_node)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

In [ ]:
count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print("Pagerank Random Walk with No teleportation, No TF-IDF ")
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank Random Walk with No teleportation, No TF-IDF 
unvisited =  0
              precision    recall  f1-score   support

           0       0.31      0.52      0.39       285
           1       0.47      0.52      0.49       406
           2       0.57      0.27      0.36       726
           3       0.54      0.52      0.53       379
           4       0.14      0.19      0.16       214
           5       0.10      0.22      0.14       131
           6       0.24      0.18      0.20       344

    accuracy                           0.35      2485
   macro avg       0.34      0.35      0.32      2485
weighted avg       0.41      0.35      0.36      2485



0.35492957746478876

In [ ]:
#pagerank. WITH telportation, without tfidf 

#repeat above experiment but this time use the teleportation random walk 

#get metrics
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

telep_prob = [0.1]

for train_node,predicted in zip(train['node'],train['label']):
    for tp in telep_prob:
  #print (train_node,predicted)
        for i in range(num_samples):
            start_point = train_node
            end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
    if vf ==0:
        count+=1

print("Pagerank Random Walk with teleportation, No TF-IDF ")
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank Random Walk with teleportation, No TF-IDF 
unvisited =  18
              precision    recall  f1-score   support

           0       0.68      0.78      0.73       285
           1       0.78      0.89      0.83       406
           2       0.83      0.57      0.68       726
           3       0.79      0.74      0.77       379
           4       0.67      0.82      0.74       214
           5       0.42      0.82      0.55       131
           6       0.64      0.61      0.62       344

    accuracy                           0.71      2485
   macro avg       0.69      0.75      0.70      2485
weighted avg       0.74      0.71      0.71      2485



0.7118712273641851

In [ ]:
#pagerank. WITH telportation, without tfidf 

#repeat above experiment but this time use the teleportation random walk 

#get metrics
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

telep_prob = [0.2]

for train_node,predicted in zip(train['node'],train['label']):
    for tp in telep_prob:
  #print (train_node,predicted)
        for i in range(num_samples):
            start_point = train_node
            end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
    if vf ==0:
        count+=1

print("Pagerank Random Walk with teleportation, No TF-IDF ")
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank Random Walk with teleportation, No TF-IDF 
unvisited =  64
              precision    recall  f1-score   support

           0       0.59      0.79      0.68       285
           1       0.81      0.86      0.83       406
           2       0.82      0.58      0.68       726
           3       0.80      0.72      0.76       379
           4       0.65      0.79      0.71       214
           5       0.40      0.76      0.53       131
           6       0.64      0.59      0.61       344

    accuracy                           0.70      2485
   macro avg       0.67      0.73      0.69      2485
weighted avg       0.73      0.70      0.70      2485



0.6985915492957746

In [ ]:
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

#f1 anf f2 comes from the tfiDF

for n1 in nodes:
    for n2 in nodes:
    # if there is an edge between n1 and n2:
        if G.has_edge(n1,n2):
            f1 = list(df['features'])[n1]
            f2 = list(df['features'])[n2]
      # cos_sim = compute cosine similarity between features of n1 and n2
            cos_sim = np.dot(f1, f2)/(norm(f1)*norm(f2))
            transition[n1,n2] = np.exp(cos_sim) #neumerator of softmax. #why do we need softmax?  
#divide the values in transition by denominator of softmax. how will you do this? 

transition = transition/transition.sum(axis=1, keepdims=True) 


In [ ]:
#pagerank. Without teleportation. WITH TFIDF 
transition_matrix = transition

#perfrom pagerank using our tf_idf based transition matrix
#use randon walk without teleporation 
#get metrics
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]


for train_node,predicted in zip(train['node'],train['label']):
  #print (train_node,predicted)
  for i in range(num_samples):
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      end_node = int(end_node)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf == 0:
    count+=1

print("Pagerank Random Walk with No teleportation, with TF-IDF ")
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank Random Walk with No teleportation, with TF-IDF 
unvisited =  0
              precision    recall  f1-score   support

           0       0.32      0.53      0.40       285
           1       0.44      0.48      0.46       406
           2       0.49      0.25      0.33       726
           3       0.66      0.58      0.62       379
           4       0.12      0.16      0.14       214
           5       0.08      0.18      0.11       131
           6       0.27      0.23      0.25       344

    accuracy                           0.36      2485
   macro avg       0.34      0.35      0.33      2485
weighted avg       0.41      0.36      0.37      2485



0.3573440643863179

In [ ]:
#pagerank. WITH teleportation WITH TFIDF
transition_matrix = transition

#same as above, except use random walk with teleportation 
#get metrics
num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

telep_prob = [0.1]

for train_node,predicted in zip(train['node'],train['label']):
    for tp in telep_prob:
  #print (train_node,predicted)
        for i in range(num_samples):
            start_point = train_node
            end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1

print("Pagerank Random Walk with teleportation and TF-IDF ")
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

Pagerank Random Walk with teleportation and TF-IDF 
unvisited =  21
              precision    recall  f1-score   support

           0       0.67      0.79      0.72       285
           1       0.80      0.89      0.84       406
           2       0.84      0.58      0.69       726
           3       0.79      0.75      0.77       379
           4       0.68      0.81      0.74       214
           5       0.42      0.82      0.55       131
           6       0.65      0.61      0.63       344

    accuracy                           0.72      2485
   macro avg       0.69      0.75      0.71      2485
weighted avg       0.74      0.72      0.72      2485



0.7175050301810865

In [ ]:
#pagerank. WITH teleportation WITH TFIDF
transition_matrix = transition

#same as above, except use random walk with teleportation 
#get metrics
#pagerank. WITH telportation, without tfidf 

#repeat above experiment but this time use the teleportation random walk 

#get metrics

#pagerank. NO teleportation, NO tfidf. 
transition_matrix = create_transition_matrix(G)

num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

telep_prob = [0.2]

for train_node,predicted in zip(train['node'],train['label']):
    for tp in telep_prob:
  #print (train_node,predicted)
        for i in range(num_samples):
            start_point = train_node
            end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted, transition_matrix)
            end_node = int(end_node)
            visiting_freq_label[end_node][predicted] += 1
            visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
accuracy_score(df['label'], preds)

unvisited =  58
              precision    recall  f1-score   support

           0       0.61      0.79      0.69       285
           1       0.80      0.88      0.84       406
           2       0.82      0.58      0.68       726
           3       0.80      0.73      0.76       379
           4       0.68      0.78      0.73       214
           5       0.42      0.79      0.55       131
           6       0.65      0.59      0.62       344

    accuracy                           0.71      2485
   macro avg       0.68      0.74      0.69      2485
weighted avg       0.73      0.71      0.71      2485



0.7058350100603622